In [2]:
import numpy as np
from numpy import random
import random
import matplotlib.pyplot as plt
from scipy.signal import cwt,ricker
from scipy import stats
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import os
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import load_model
from tensorflow.keras.layers import Flatten,concatenate,Layer,Dense,BatchNormalization
import radiomics
from radiomics.featureextractor import RadiomicsFeatureExtractor
import SimpleITK as sitk
import pandas as pd


# Example: Load An Image

Example how to load an image with label

In [100]:
# the folder name by label
class_names = ['pos','neg']

# the image dictionary we are loading
dict = './covid19/kcv/'

In [101]:
# load one image with their label to memory
def load_image( imageName ):

    for i in class_names:
        path = os.path.join(dict, i)
        label_num = class_names.index(i)
        try:
            img = sitk.ReadImage(dict + i +'/' + imageName)
            trainingdata= [sitk.GetArrayFromImage(img).reshape(512,512,1),label_num]
        except:
            pass
            
    return trainingdata
    

In [102]:
singleImage = load_image('COVID-19_0414.png')

In [103]:
# get the label and the image
image = singleImage[0]
label = singleImage[1]

In [104]:
print(label)

0


# Example: Extract the Features From a Image

Example how to extract the Features using pyradiomics library applied in a whole image

In [79]:
# the image dictionary and the image name want to extract feature from
dict = './covid19/kcv/pos/'

def ExtractFeaturesFromImages(imageName):
    
    feature_extracted = []

    extractor = RadiomicsFeatureExtractor()
    # Disable all classes 
    extractor.disableAllFeatures()
    extractor.enableFeaturesByName(firstorder=['Skewness'])  # change here to extract

    im = sitk.ReadImage(dict + imageName)

    ma_arr = np.ones(im.GetSize()[::-1])  # reverse the order as image is xyz, array is zyx
    ma = sitk.GetImageFromArray(ma_arr)
    ma.CopyInformation(im)  # Copy geometric info

    # extract features and fill the output array
    features = extractor.execute(im, ma)

    feature_extracted.append(features['original_firstorder_Skewness'])
      
    return feature_extracted


In [80]:
feature_extracted = ExtractFeaturesFromImages('COVID-19_0414.png')

# Example: To Train A Fin

When we have images and corrresponding feature extracted, we can train a DFNN to iminate these features

In [ ]:

def make_untrained_FIN(topology,input_dim, name="my-FIN"):
    input_layer = keras.Input(shape=input_dim)
    x = Flatten()(input_layer)

    for layer in topology['layers']:
        x = Dense(layer['output_shape'], activation='relu')(x) 
        if layer.get('BatchNorm', False):
            x = BatchNormalization()(x)

    output_layer = Dense(1, activation='linear')(x)
    fin = keras.Model(inputs=input_layer, outputs=output_layer, name=name)
    
    return fin

In [ ]:
model = make_untrained_FIN('Name')

In [ ]:
topology = np.load('hyper_parameters.npy', allow_pickle=True)[()] # can use with random topology generated
# Configure the optimizer
optimizer = tf.keras.optimizers.Adam(
        learning_rate=topology['lr'], 
        beta_1=topology['b1'], 
        beta_2=topology['b2'])
        
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [ ]:
model.fit(input_array, output_array, 
              epochs=30, 
          batch_size=64,
                shuffle=True
            )

# Example: Load A Fin and Apply in a Image

Example we load an image into numpy array, and we use Fin to predict the feature of the image

In [95]:
image = np.array(image)

In [96]:
print(image.shape)

(512, 512, 1)


In [97]:
fin = load_model('finetune_Autocorrelation.h5')

In [87]:
inputimage = image.reshape(1,512,512,1)

In [99]:
fin.predict(inputimage)

1/1 [==============================] - 0s 30ms/step


array([[42.890038]], dtype=float32)

# Example Use the FINs in another Network

Load the FINs and attached to a DFNN

In [ ]:
# Load two Fins
fin1 = load_model('finetune_Skewness.h5')
fin2 = load_model('finetune_Autocorrelation.h5')


def MakeModel(modelname):
    input_layer = keras.Input(shape=(512,512,1))
    
    # call and concat based on actual usage
    l1 = fin1(input_layer)
    l2 = fin2(input_layer)
    
    n_d = concatenate([l1,l2])
    n_d_norm = BatchNormalization()(n_d) # add batch normalization, also min-max to 0-1 for preprocessing is preferred

    x = Dense(128, activation='relu',name='x1')(n_d_norm)

    out = Dense(2, activation = 'softmax', name= modelname)(x)

    model = keras.Model(input_layer, out, name= modelname)
    return model